# NLP: #1: Strings, distances, regexp

In [111]:
# --- edit distances ---

def hemming_distance(s1, s2):
    # assert len(s1) == len(s2), "strings must be the same length"
    d = 0
    for xi, xj in zip(s1, s2):
        d += xi != xj
    return d

# inv
def jaro_distance(s1, s2):
    s1_len = len(s1)
    s2_len = len(s2)
    d_max = round(max(s1_len, s2_len)/2) - 1
    m = 0
    t = 0
    for i in range(s1_len):
        for j in range(max(0, i - d_max), min(s2_len, i + d_max)):
            if s1[i] == s2[j]:
                m += 1
                if i != j:
                    t += 1
    t //= 2
    if m == 0:
        return 0
    
    return 1/3 * (m/s1_len + m/s2_len + (m-t)/m)

# inv
def jaro_winkler_distance(s1, s2):
    def max_prefix_length(s1_, s2_):
        l = 0
        for xi, xj in zip(s1_, s2_):
            if xi != xj:
                return l
            l += 1
        return l
    l = max_prefix_length(s1, s2)
    p = 0.1
    dj = jaro_distance(s1, s2)
    dw = dj + l * p * (1 - dj)
    return dw
    

def levenshtein_distance(s1, s2):
    n, m = len(s1), len(s2)
    if n > m:
        s1, s2 = s2, s1
        n, m = m, n

    current_row = range(n + 1)
    for i in range(1, m + 1):
        previous_row, current_row = current_row, [i] + [0] * n
        for j in range(1, n + 1):
            add, delete, change = previous_row[j] + 1, current_row[j - 1] + 1, previous_row[j - 1]
            if s1[j - 1] != s2[i - 1]:
                change += 1
            current_row[j] = min(add, delete, change)

    return current_row[n]

def _lcs(X, Y, m, n): 
    if m == 0 or n == 0: 
        return 0
    elif X[m-1] == Y[n-1]: 
        return 1 + _lcs(X, Y, m-1, n-1)
    else: 
        return max(_lcs(X, Y, m, n-1), _lcs(X, Y, m-1, n))

def lcs(s1, s2):
    m, n = len(s1), len(s2)
    return _lcs(s1, s2, m, n)

# --- * ---

def jac_distance(s1, s2):
    set_s1 = set(s1)
    set_s2 = set(s2)
    intersection_len = len(set_s1.intersection(set_s2))
    return intersection_len/(len(set_s1) + len(set_s2) - intersection_len)

In [112]:
import numpy as np
import pandas as pd
import textdistance
data = list()

cases = [
    ("apple", "orange"),
    ("ivan", "fedot"),
    ("apple", "bapple"),
    ("foo", "fizz"),
    ("bar", "bare"),
    ("abcabc", "abcdabcd"),
    ("aabcdef", "bcdeeef"),
    ("like", "like")
]

for s1, s2 in cases:
    current = [s1, s2]
    hem1 = hemming_distance(s1, s2)
    hem2 = textdistance.hamming(s1, s2)
    jaro1 = jaro_distance(s1, s2)
    jaro2 = textdistance.jaro(s1, s2)
    jaro_winkler1 = jaro_winkler_distance(s1, s2)
    jaro_winkler2 = textdistance.jaro_winkler(s1, s2)
    lev1 = levenshtein_distance(s1, s2)
    lev2 = textdistance.levenshtein(s1, s2)
    lc1 = lcs(s1, s2)
    lc2 = len(textdistance.lcsseq(s1, s2))
    jac1 = jac_distance(s1, s2)
    jac2 = textdistance.jaccard(s1, s2)
    current.extend(
        [hem1, hem2, jaro1, jaro2, jaro_winkler1, jaro_winkler2, lev1, lev2, lc1, lc2, jac1, jac2]
    )
    data.append(current)

df = pd.DataFrame(
    data, columns=['s1', 's2',
                   'hem-1', 'hem-2',
                   'jaro-1', 'jaro-2',
                   'jaro-wi-1', 'jaro-wi-2',
                   'levenshtein-1', 'levenshtein-2',
                   'lcs-1', 'lcs-2',
                   'jacindex-1', 'jacindex-2']
)

In [113]:
df

,s1,s2,hem-1,hem-2,jaro-1,jaro-2,jaro-wi-1,jaro-wi-2,levenshtein-1,levenshtein-2,lcs-1,lcs-2,jacindex-1,jacindex-2
0,apple,orange,5,6,0.455556,0.577778,0.455556,0.577778,5,5,2,2,0.250000,0.222222
1,ivan,fedot,4,5,0.000000,0.000000,0.000000,0.000000,5,5,0,0,0.000000,0.000000
2,apple,bapple,4,5,0.955556,0.944444,0.955556,0.944444,1,1,5,5,0.800000,0.833333
3,foo,fizz,2,3,0.527778,0.527778,0.575000,0.527778,3,3,1,1,0.250000,0.166667
4,bar,bare,0,1,0.916667,0.916667,0.941667,0.916667,1,1,3,3,0.750000,0.750000
5,abcabc,abcdabcd,3,5,1.097222,0.916667,1.068056,0.941667,2,2,6,6,0.750000,0.750000
6,aabcdef,bcdeeef,5,5,0.904762,0.809524,0.904762,0.809524,4,4,5,5,0.833333,0.555556
7,like,like,0,0,1.000000,1.000000,1.000000,1.000000,0,0,4,4,1.000000,1.000000
